# Image Segmentation with U-Net
In this lab you will learn how to build and train an image segmentation model with `tf.keras` and (optionally) train at scale with Vertex AI.

Learning Objectives:
* Learn how to preprocess and augment data for image segmentation
* Learn how to build a U-net architecture with `tf.keras`
* Learn how to train this model with Vertex AI
* Learn how to serve and parse predictions 

## Setup

In [ ]:
import os
import warnings

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")

In [ ]:
import time
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

tf.keras.backend.clear_session()

PROJECT = !(gcloud config get-value project)
PROJECT = PROJECT[0]
REGION = "us-central1"
BUCKET = PROJECT
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
OUTPUT_DIR = f"gs://{BUCKET}/pets/models/{TIMESTAMP}"

# silence TF info logs
%env TF_CPP_MIN_LOG_LEVEL=3

print(tf.__version__)

#### Visualize the data
The dataset used in this lab in the [Oxford Pet Dataset](https://www.tensorflow.org/datasets/catalog/oxford_iiit_pet). It contains images of animals with segmentation masks. The goal of an image segmentation model is to take in as input an image and predict the segmentation mask. Think multi-classification problem, where each pixel is labelled as background, interior, or outline. 

In [ ]:
def read_and_preprocess(data):
    input_image = tf.image.resize(data["image"], (128, 128))
    input_mask = tf.image.resize(data["segmentation_mask"], (128, 128))

    input_image = tf.image.convert_image_dtype(input_image, tf.float32)  # [0,1]
    input_mask -= 1  # {1,2,3} to {0,1,2}
    return input_image, input_mask

In [ ]:
AUTO = tf.data.AUTOTUNE

# Load in the dataset with tfds
dataset, info = tfds.load(
    "oxford_iiit_pet",
    data_dir="gs://asl-public/data/tensorflow_datasets",
    with_info=True,
)

train = dataset["train"].map(read_and_preprocess, num_parallel_calls=AUTO)
test = dataset["test"].map(read_and_preprocess, num_parallel_calls=AUTO)

f, ax = plt.subplots(2, 5, figsize=(16, 5))
for idx, (img, mask) in enumerate(train.take(5)):
    ax[0, idx].imshow(tf.keras.preprocessing.image.array_to_img(img))
    ax[0, idx].axis("off")
    mask = tf.reshape(mask, [128, 128])
    ax[1, idx].imshow(mask.numpy())
    ax[1, idx].axis("off")

#### Data Augmentation
Create a function that randomly flips the image and mask left or right.

In [ ]:
AUGMENT_PROB = 0.5  # probability of data augmentation


def augment(img, mask):
    if tf.random.uniform(()) < AUGMENT_PROB:
        img = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)
    return img, mask


train = train.map(augment)
test = test.map(augment)

f, ax = plt.subplots(2, 5, figsize=(16, 5))
for idx, (img, mask) in enumerate(train.take(5)):
    ax[0, idx].imshow(tf.keras.preprocessing.image.array_to_img(img))
    ax[0, idx].axis("off")
    mask = tf.reshape(mask, [128, 128])
    ax[1, idx].imshow(mask.numpy())
    ax[1, idx].axis("off")

### Modified U-Net

The model being used here is a modified [U-Net](https://arxiv.org/pdf/1505.04597v1.pdf). A U-Net typically consists of an encoder which downsamples an image to an encoding, and a mirrored decoder which upsamples the encoding back to the desired mask. The decoder blocks have a number of skip connections that directly connect the encoder blocks to the decoder.

In this notebook, we will use a pretrained MobileNetV2 to create the encoding and a set of upsampling layers (stride=2) to get back to the desired mask. When doing so, however, we will pull out layers with the desired sizes so that the upsampling skip layers use corresponding weights from the pretrained models. The layer names such as `block_1_expand_relu`, `block_3_expand_relu`, etc, correspond to these output layers of the pretrained MobileNetV2. If you would like to see the entire structure of the pretrained MobileNetV2, feel free to load it into memory and analyze it with `tf.keras.utils.plot_model` or `model.summary()`

Note that the output will consist of 3 channels. This is because we have 3 possible labels for each pixel (background, outline, interior). Think of this as multi-classification problem with three possible classes.

![image](https://storage.googleapis.com/asl-public/public-images/unet.png)
Architecture diagram from original research paper

In [ ]:
# Specify shape and channels of input images
INPUT_SHAPE = [128, 128, 3]
OUTPUT_CHANNELS = 3

#### Create the Encoder (Downstack)

First, create an encoder which downsamples the input images. To keep things simple, use a pretrained model (MobileNetV2). Set `include_top=False` since you don't want to include the fully-connected layer at the top of the network. 

In [ ]:
# load model into memory
base_model = tf.keras.applications.MobileNetV2(
    input_shape=INPUT_SHAPE, include_top=False
)

Recall that Unet uses outputs of the encoder to form skip connections to the decoder. Access these layers by name to create the output of the encoder, then instantiate it as a `tf.keras.Model`.

To make this architecture train faster you can freeze the weights of the encoder so they are not learned at training time. You can do this by setting the `trainable` attibute to `False` on the `tf.keras.Model`.

In [ ]:
# Use the activations of these layers to form skip connections
layer_names = [
    "block_1_expand_relu",  # 64x64
    "block_3_expand_relu",  # 32x32
    "block_6_expand_relu",  # 16x16
    "block_13_expand_relu",  # 8x8
    "block_16_project",  # 4x4
]

# Outputs
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(
    inputs=base_model.input,
    outputs=base_model_outputs,
    name="pretrained_mobilenet",
)

down_stack.trainable = False

print(f"Encoder Inputs:\n{down_stack.inputs}")
print("Encoder Outputs:")
for i in range(len(down_stack.outputs)):
    print(down_stack.outputs[i])

As you can see the encoder is expecting an image of height/width `[128, 128]` as input and is downsampling to `[64, 64]`, `[32, 32]`, `[16, 16]`, `[8, 8]`, and `[4, 4]`.

#### Create the Decoder (Upstack)

First, create a function that returns upsampling layers as simple `tf.keras.Sequential` models. Remember that the upsampling layers should double the image sizes. To do this we can use a [transposed convolutional layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose) with `strides=2` and `padding="same"`. 

In [ ]:
def upsample(filters, kernel_size, name, strides=2):
    # TODO: Write the upsample function to perform upsamples, batch normalization, and activation
    # This function should return a tf.keras.Sequential model

Use the upsample function to create the upsampling layers that bring the dimensionality back up.

In [ ]:
up_stack = [
    upsample(512, 3, "upsample_4x4_to_8x8"),
    upsample(256, 3, "upsample_8x8_to_16x16"),
    upsample(128, 3, "upsample_16x16_to_32x32"),
    upsample(64, 3, "upsample_32x32_to_64x64"),
]

up_stack

Send inputs through the encoder, create and establish the skip connections while upsampling through the decoder. 

In [ ]:
inputs = tf.keras.layers.Input(shape=INPUT_SHAPE, name="input_image")

# Downsampling through the model
skips = down_stack(inputs)
x = skips[-1]
skips = reversed(skips[:-1])

for idx, (up, skip) in enumerate(zip(up_stack, skips)):
# TODO: Implement the upsampling and establish skip connections 

The upstack as we have it now outputs with height/width `[64, 64]`. Create the last layer manually, which will be a `Conv2DTranspose` to upsample back to the original input shape. Then, instatiate the end-to-end architecture as a `tf.keras.Model`.

In [ ]:
# This is the last layer of the model
last = tf.keras.layers.Conv2DTranspose(
    OUTPUT_CHANNELS, 3, strides=2, padding="same"
)  # 64x64 -> 128x128

x = last(x)

# TODO: Initialize and compile the model

tf.keras.utils.plot_model(model, show_shapes=True)

As you can see, the input is an image with shape `[128, 128, 3]`. This image gets downsampled using the pretrained MobileNetV2, then upsampled back to it's original size with `Conv2DTranspose`. Also, note the skip connections, critical to the Unet architecture. 

#### Train the model locally
**Note**: Without a GPU, training for 10 epochs can take ~ 45 minutes.

In [ ]:
EPOCHS = 10
BATCH_SIZE = 8
STEPS_PER_EPOCH = info.splits["train"].num_examples // BATCH_SIZE
VALIDATION_STEPS = info.splits["test"].num_examples // BATCH_SIZE
MODEL_DIR = "./trained_unet"
train = train.batch(BATCH_SIZE).repeat()
test = test.batch(BATCH_SIZE)

In [ ]:
history = model.fit(
    train,
    validation_data=test,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
)

In [ ]:
pd.DataFrame(history.history)[["val_accuracy", "accuracy"]].plot(
    title="Accuracy"
)
pd.DataFrame(history.history)[["val_loss", "loss"]].plot(title="Loss");

#### Visualize predictions
These are helper functions to visualize the predictions of the trained model. 

`create_mask`: Returns the mask assigning each pixel to the 'class' with the highest probabilty (background, outline, interior). 

`display`: Plots an input image, the true mask (label) of the image, and the predicted mask from our model.

`show_prediction`: Calls `display` for a specified number of input images.

In [ ]:
def create_mask(pred_mask):
    """Given prediction mask, return highest probability per pixel"""
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


def display(display_list):
    """Plot input image, true mask, and predicted mask"""
    plt.figure(figsize=(15, 15))
    title = ["Input Image", "True Mask", "Predicted Mask"]

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis("off")
    plt.show()


def show_predictions(dataset, num):
    """Display a certain number of predictions from a tf.data.Dataset"""
    for image, mask in dataset.take(num):
        pred_mask = model.predict(image)
        display([image[0], mask[0], create_mask(pred_mask)])

In [ ]:
show_predictions(test, 5)

#### (Optional) Train on Vertex AI
In this section you containerize the training application and submit a custom training job to Vertex AI. The steps to do this are:

* Create Python script of the training application
* Create Dockerfile to bundle training script and deps
* Build and push Docker image
* Create and run Vertex AI Custom Training Job

In [ ]:
!mkdir ./pets_trainer

In [ ]:
%%writefile ./pets_trainer/train.py

import tensorflow as tf
import tensorflow_datasets as tfds
import fire

def read_and_preprocess(data):
    input_image = tf.image.resize(data['image'], (128,128))
    input_mask = tf.image.resize(data['segmentation_mask'], (128,128))
    
    input_image = tf.image.convert_image_dtype(input_image, tf.float32) # [0,1]
    input_mask -= 1 # {1,2,3} to {0,1,2}
    return input_image, input_mask
    
def augment(img, mask):
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)
    return img, mask
    
def create_train_test_datasets(dataset, batch_size):
    train_ds = dataset['train'].map(read_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = dataset['test'].map(read_and_preprocess)
    train_ds = train_ds.cache().map(augment).shuffle(10 * batch_size).batch(batch_size).repeat()
    train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    test_ds = test_ds.batch(batch_size)
    return train_ds, test_ds
    
def upsample(filters, size, name):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU()], name=name)

def build_unet(input_shape, output_channels):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False)
    
    # Use the activations of these layers to form skip connections
    layer_names = [
        'block_1_expand_relu',   # 64x64
        'block_3_expand_relu',   # 32x32
        'block_6_expand_relu',   # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',      # 4x4
    ]
    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]
    
    # Create the feature extraction model
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs,
                                name='pretrained_mobilenet')

    down_stack.trainable = False
    
    up_stack = [
        upsample(512, 3, 'upsample_4x4_to_8x8'),
        upsample(256, 3, 'upsample_8x8_to_16x16'),
        upsample(128, 3, 'upsample_16x16_to_32x32'),
        upsample(64, 3,  'upsample_32x32_to_64x64')
    ]
    
    inputs = tf.keras.layers.Input(shape=input_shape, name='input_image')
    
    # Downsampling through the model
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])
    
    # Upsampling and establishing the skip connections
    for idx, (up, skip) in enumerate(zip(up_stack, skips)):
        x = up(x)
        concat = tf.keras.layers.Concatenate(name='expand_{}'.format(idx))
        x = concat([x, skip])
        
    # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
        output_channels, 3, strides=2,
        padding='same')  #64x64 -> 128x128
    
    x = last(x)
    return tf.keras.Model(inputs=inputs, outputs=x)
    
    
def train_evaluate(
    batch_size=64,
    tf_dataset='oxford_iiit_pet',
    data_dir='gs://asl-public/data/tensorflow_datasets',
    input_shape=[128,128,3],
    output_channels=3,
    epochs=10,
    output_dir=None
):
    
    # Download dataset from tfds
    dataset, info = tfds.load(tf_dataset, data_dir=data_dir, with_info=True)
    
    # Create train and test datasets
    train_ds, test_ds = create_train_test_datasets(dataset, batch_size)
    
    steps_per_epoch = info.splits['train'].num_examples // batch_size
    validation_steps = info.splits['test'].num_examples // batch_size
    
    model = build_unet(input_shape, output_channels)
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    
    history = model.fit(
        train_ds, 
        validation_data=test_ds,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    if output_dir:
        model.save(output_dir)
        
if __name__ == '__main__':
    fire.Fire(train_evaluate)

#### Write Dockerfile

In [ ]:
%%writefile ./pets_trainer/Dockerfile
FROM us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-12.py310:latest

# Installs fire
RUN pip install -U fire tensorflow_datasets

# Copies the trainer code to the docker image.
WORKDIR /app
COPY train.py .

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "train.py"]

#### Build and push containerized training application

In [ ]:
IMAGE_NAME = "pets_trainer_tf"
TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT}/{IMAGE_NAME}:{TAG}"
TRAINING_CONTAINER_IMAGE_URI

In [ ]:
!gcloud builds submit --machine-type=e2-highcpu-32 --timeout=15m --tag $TRAINING_CONTAINER_IMAGE_URI pets_trainer

#### Run Custom Training Job on Vertex AI 

Check to make sure bucket exists and if not create it

In [ ]:
STAGING_BUCKET = f"gs://{BUCKET}-staging"

%env STAGING_BUCKET={STAGING_BUCKET}
%env REGION={REGION}

In [ ]:
%%bash
exists=$(gsutil ls -d | grep -w ${STAGING_BUCKET}/)

if [ -n "$exists" ]; then
   echo -e "Bucket gs://${BUCKET} already exists."
    
else
   echo "Creating a new GCS bucket."
   gsutil mb -l ${REGION} ${STAGING_BUCKET}
   echo -e "\nHere are your current buckets:"
   gsutil ls
fi

In [ ]:
# Specify variables used to configure hyperparameter tuning job
DISPLAY_NAME = "pets-segmentation"
JOB_NAME = f"unet-custom-training-{TIMESTAMP}"
EPOCHS = 50

In [ ]:
from google.cloud import aiplatform

# Define required specifications
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-8",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAINING_CONTAINER_IMAGE_URI,
            "args": [f"--output_dir={OUTPUT_DIR}", f"--epochs={EPOCHS}"],
        },
    }
]

custom_job = aiplatform.CustomJob(
    display_name=JOB_NAME,
    project=PROJECT,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

In [ ]:
custom_job.run()

Congrats! You've succesfully trained a custom U-net model for image segmentation and operationalized model training with Vertex AI.